Basic imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    KFold,
    cross_val_score,
    train_test_split,
    GridSearchCV,
    StratifiedKFold,
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier

from bayes_opt import BayesianOptimization
from skopt import BayesSearchCV
import lightgbm as lgb

warnings.filterwarnings("ignore")
%load_ext nb_black

RANDOM_STATE = 7

plt.style.use("ggplot")

<IPython.core.display.Javascript object>

Importing data

In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
under_train_df = pd.read_csv("train_under.csv")

<IPython.core.display.Javascript object>

### 2-5 Modeling.

In [20]:
# extracting X and y from dataframes (unbalanced dataset)
X = train_df.iloc[:, 2:].values
y = train_df.TARGET.values

# extracting X and y from dataframes (balanced dataset)
X_under = under_train_df.iloc[:, 1:].values
y_under = under_train_df.TARGET.values

# extracting X from test dataframe
X_val = test_df.iloc[:, 1:].values

<IPython.core.display.Javascript object>

In [21]:
# splitting X and y to train and test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=RANDOM_STATE
)

X_train_under, X_test_under, y_train_under, y_test_under = train_test_split(
    X_under, y_under, test_size=0.2, random_state=RANDOM_STATE
)

<IPython.core.display.Javascript object>

**2-5-1** Building baseline model - Dummy Classifier and finding its performance.

In [5]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)

y_pred = dummy_clf.predict(X_test)

<IPython.core.display.Javascript object>

In [6]:
def clf_performance_metrics(y_test: np.array, y_pred: np.array) -> tuple:
    """By giving y_test and y_pred, returns classifiers accuracy, precision, recall, f1"""
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return accuracy, precision, recall, f1

<IPython.core.display.Javascript object>

In [7]:
perf_metrics = clf_performance_metrics(y_test, y_pred)
perf_metrics

(0.9192486797263196, 0.0, 0.0, 0.0)

<IPython.core.display.Javascript object>

In [8]:
y_proba = dummy_clf.predict_proba(X_val)

<IPython.core.display.Javascript object>

In [9]:
def to_csv(y_pred_proba: list, csv_name: str) -> None:
    """Saves predictions to csv file with specified name"""
    df = pd.DataFrame()
    df["SK_ID_CURR"] = test_df.SK_ID_CURR
    df["TARGET"] = y_pred_proba
    df.to_csv(csv_name + ".csv", index=False)

<IPython.core.display.Javascript object>

In [10]:
to_csv(y_proba[:, 1], "dummy")

<IPython.core.display.Javascript object>

**2-5-2** Creating dataframe to keep tracking information about model performance.

In [11]:
models_df = pd.DataFrame(
    columns=[
        "model",
        "balanced_dataset",
        "features",
        "hyperparameters",
        "accuracy",
        "precision",
        "recall",
        "f1",
        "Kaggle",
    ]
)

<IPython.core.display.Javascript object>

In [12]:
new_row = {
    "model": "DummyClassifier",
    "balanced_dataset": False,
    "features": "All",
    "hyperparameters": None,
    "accuracy": perf_metrics[0],
    "precision": perf_metrics[1],
    "recall": perf_metrics[2],
    "f1": perf_metrics[3],
    "Kaggle": 0.5,
}
models_df = models_df.append(new_row, ignore_index=True)

<IPython.core.display.Javascript object>

**2-5-3** Building basic models without selector and hyperparameters tuning on unbalanced dataset.

In [13]:
# 5 models will be used: LogisticReg, RFC, KNNC, LightGBM
models = [
    LogisticRegression(random_state=RANDOM_STATE),
    RandomForestClassifier(random_state=RANDOM_STATE),
    KNeighborsClassifier(),
    LGBMClassifier(random_state=RANDOM_STATE),
]
model_names = ["logreg", "rfc", "knn", "lgbm"]

for counter in range(len(models)):
    # training model
    model = models[counter]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # measuring performance
    perf_metrics = clf_performance_metrics(y_test, y_pred)
    y_proba = model.predict_proba(X_val)

    # saving to csv
    to_csv(y_proba[:, 1], "kaggle_" + model_names[counter] + str(1))

    # exporting performance data to models_df
    new_row = {
        "model": model_names[counter],
        "balanced_dataset": False,
        "features": "All",
        "hyperparameters": None,
        "accuracy": perf_metrics[0],
        "precision": perf_metrics[1],
        "recall": perf_metrics[2],
        "f1": perf_metrics[3],
        "Kaggle": None,
    }
    models_df = models_df.append(new_row, ignore_index=True)

<IPython.core.display.Javascript object>

In [14]:
# updating Kaggle scores
models_df.iloc[1:5, -1] = [0.7539, 0.7174, 0.5871, 0.7731]

<IPython.core.display.Javascript object>

In [15]:
models_df[models_df.balanced_dataset == False]

,model,balanced_dataset,features,hyperparameters,accuracy,precision,recall,f1,Kaggle
0,DummyClassifier,False,All,None,0.919249,0.000000,0.000000,0.000000,0.5
1,logreg,False,All,None,0.919522,0.520154,0.043653,0.080547,0.7539
2,rfc,False,All,None,0.919418,0.551181,0.011276,0.022099,0.7174
3,knn,False,All,None,0.911535,0.218424,0.037049,0.063352,0.5871
4,lgbm,False,All,None,0.920003,0.590625,0.030445,0.057904,0.7731


<IPython.core.display.Javascript object>

LGBM won the fight so far, however score needs to be improved.

**2-5-4** Building basic models without selector and hyperparameters tuning on balanced dataset.

In [16]:
# 4 models will be used: LogisticReg, RFC, KNNC, LightGBM
models = [
    LogisticRegression(random_state=RANDOM_STATE),
    RandomForestClassifier(random_state=RANDOM_STATE),
    KNeighborsClassifier(),
    LGBMClassifier(random_state=RANDOM_STATE),
]

for counter in range(len(models)):
    # training model
    model = models[counter]
    model.fit(X_train_under, y_train_under)
    y_pred = model.predict(X_test_under)

    # measuring performance
    perf_metrics = clf_performance_metrics(y_test_under, y_pred)
    y_proba = model.predict_proba(X_val)

    # saving to csv
    to_csv(y_proba[:, 1], "kaggle_bal_" + model_names[counter] + str(1))

    # exporting performance data to models_df
    new_row = {
        "model": model_names[counter],
        "balanced_dataset": True,
        "features": "All",
        "hyperparameters": None,
        "accuracy": perf_metrics[0],
        "precision": perf_metrics[1],
        "recall": perf_metrics[2],
        "f1": perf_metrics[3],
        "Kaggle": None,
    }
    models_df = models_df.append(new_row, ignore_index=True)

<IPython.core.display.Javascript object>

In [17]:
# updating Kaggle scores
models_df.iloc[5:, -1] = [0.7591, 0.7358, 0.6452, 0.7707]

<IPython.core.display.Javascript object>

In [18]:
models_df[models_df.balanced_dataset == True]

,model,balanced_dataset,features,hyperparameters,accuracy,precision,recall,f1,Kaggle
5,logreg,True,All,None,0.701309,0.708428,0.689116,0.698638,0.7591
6,rfc,True,All,None,0.682175,0.686167,0.677090,0.681598,0.7358
7,knn,True,All,None,0.611078,0.607109,0.640208,0.623220,0.6452
8,lgbm,True,All,None,0.713293,0.715320,0.713169,0.714243,0.7707


<IPython.core.display.Javascript object>

Kaggle score improved: RFC, KNN, Logistic Regression

Kaggle score dropped: LightGBM

Still need to improve it.

**2-5-5** Models with features selectors on unbalanced dataset. KNeighborsClassifier does not work with the selector.

In [22]:
# 3 models will be used: LogisticReg, RFC, LightGBM
models = [
    LogisticRegression(random_state=RANDOM_STATE),
    RandomForestClassifier(random_state=RANDOM_STATE),
    LGBMClassifier(random_state=RANDOM_STATE),
]
model_names = ["logreg", "rfc", "lgbm"]

for counter in range(len(models)):
    # training selector
    try:
        selector = SelectFromModel(models[counter])
        selector.fit(X_train, y_train)
    except:
        selector = SelectFromModel(LogisticRegression())
        selector.fit(X_train, y_train)

    # training model
    model = models[counter]
    model.fit(X_train[:, selector.get_support()], y_train)
    y_pred = model.predict(X_test[:, selector.get_support()])

    # measuring performance
    perf_metrics = clf_performance_metrics(y_test, y_pred)
    y_proba = model.predict_proba(X_val[:, selector.get_support()])

    # saving to csv
    to_csv(y_proba[:, 1], "kaggle_sel_un_" + model_names[counter] + str(1))

    # exporting performance data to models_df
    new_row = {
        "model": model_names[counter],
        "balanced_dataset": False,
        "features": "Selected",
        "hyperparameters": None,
        "accuracy": perf_metrics[0],
        "precision": perf_metrics[1],
        "recall": perf_metrics[2],
        "f1": perf_metrics[3],
        "Kaggle": None,
    }
    models_df = models_df.append(new_row, ignore_index=True)

<IPython.core.display.Javascript object>

In [23]:
# updating Kaggle scores
models_df.iloc[9:, -1] = [0.7554, 0.7124, 0.7733]

<IPython.core.display.Javascript object>

In [24]:
models_df[models_df.features == "Selected"]

,model,balanced_dataset,features,hyperparameters,accuracy,precision,recall,f1,Kaggle
9,logreg,False,Selected,None,0.919600,0.528067,0.040915,0.075946,0.7554
10,rfc,False,Selected,None,0.919444,0.559055,0.011437,0.022415,0.7124
11,lgbm,False,Selected,None,0.919860,0.569733,0.030928,0.058671,0.7733


<IPython.core.display.Javascript object>

**2-5-6** Models with features selectors on balanced dataset.

In [25]:
# 3 models will be used: LogisticReg, RFC, LightGBM
models = [
    LogisticRegression(random_state=RANDOM_STATE),
    RandomForestClassifier(random_state=RANDOM_STATE),
    LGBMClassifier(random_state=RANDOM_STATE),
]
model_names = ["logreg", "rfc", "lgbm"]

for counter in range(len(models)):
    # training selector
    try:
        selector = SelectFromModel(models[counter])
        selector.fit(X_train_under, y_train_under)
    except:
        selector = SelectFromModel(LogisticRegression())
        selector.fit(X_train_under, y_train_under)

    # training model
    model = models[counter]
    model.fit(X_train_under[:, selector.get_support()], y_train_under)
    y_pred = model.predict(X_test_under[:, selector.get_support()])

    # measuring performance
    perf_metrics = clf_performance_metrics(y_test_under, y_pred)
    y_proba = model.predict_proba(X_val[:, selector.get_support()])

    # saving to csv
    to_csv(y_proba[:, 1], "kaggle_sel_bal_" + model_names[counter] + str(1))

    # exporting performance data to models_df
    new_row = {
        "model": model_names[counter],
        "balanced_dataset": True,
        "features": "Selected",
        "hyperparameters": None,
        "accuracy": perf_metrics[0],
        "precision": perf_metrics[1],
        "recall": perf_metrics[2],
        "f1": perf_metrics[3],
        "Kaggle": None,
    }
    models_df = models_df.append(new_row, ignore_index=True)

<IPython.core.display.Javascript object>

In [26]:
# updating Kaggle scores
models_df.iloc[12:15, -1] = [0.7574, 0.7407, 0.7706]

<IPython.core.display.Javascript object>

In [27]:
models_df[(models_df.features == "Selected") & (models_df.balanced_dataset == True)]

,model,balanced_dataset,features,hyperparameters,accuracy,precision,recall,f1,Kaggle
12,logreg,True,Selected,None,0.699799,0.707781,0.685508,0.696467,0.7574
13,rfc,True,Selected,None,0.692447,0.700103,0.678493,0.689129,0.7407
14,lgbm,True,Selected,None,0.709164,0.711496,0.708358,0.709924,0.7706


<IPython.core.display.Javascript object>

LGBM worked best without selector and with imbalanced dataset.

**2-5-7** LGBM hyperparameters tuning using Bayesian Optimization (GridSearch does not work because model is not from ScikitLearn).

In [28]:
def lgbm_evaluation(
    num_leaves,
    max_depth,
    min_split_gain,
    min_child_weight,
    min_child_samples,
    reg_alpha,
    reg_lambda,
):
    """
    Objective function for Bayesian Optimization of LightGBM's Hyperparamters. Takes the hyperparameters as input, and
    returns the Cross-Validation AUC as output.
    """

    params = {
        "objective": "binary",
        "boosting_type": "gbdt",
        "learning_rate": 0.005,
        "n_estimators": 10000,
        "n_jobs": -1,
        "num_leaves": int(round(num_leaves)),
        "max_depth": int(round(max_depth)),
        "min_split_gain": min_split_gain,
        "min_child_weight": min_child_weight,
        "min_child_samples": int(round(min_child_samples)),
        "reg_alpha": reg_alpha,
        "reg_lambda": reg_lambda,
        "verbosity": -1,
        "seed": RANDOM_STATE,
    }
    stratified_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

    cv_preds = np.zeros(X_train.shape[0])
    for train_indices, cv_indices in stratified_cv.split(X_train, y_train):

        x_tr = X_train.iloc[train_indices]
        y_tr = y_train.iloc[train_indices]
        x_cv = X_train.iloc[cv_indices]
        y_cv = y_train.iloc[cv_indices]

        lgbm_clf = lgb.LGBMClassifier(**params)
        lgbm_clf.fit(
            x_tr,
            y_tr,
            eval_set=[(x_cv, y_cv)],
            eval_metric="auc",
            verbose=False,
            early_stopping_rounds=200,
        )

        cv_preds[cv_indices] = lgbm_clf.predict_proba(
            x_cv, num_iteration=lgbm_clf.best_iteration_
        )[:, 1]

    return roc_auc_score(target_train, cv_preds)

<IPython.core.display.Javascript object>

In [29]:
bopt_lgbm = BayesianOptimization(
    lgbm_evaluation,
    {
        "num_leaves": (25, 50),
        "max_depth": (4, 11),
        "min_split_gain": (0, 0.1),
        "min_child_weight": (5, 80),
        "min_child_samples": (5, 80),
        "reg_alpha": (0.001, 0.3),
        "reg_lambda": (0.001, 0.3),
    },
    random_state=RANDOM_STATE,
)

# bayesian_optimization = bopt_lgbm.maximize(n_iter=6, init_points=4)

<IPython.core.display.Javascript object>

In [30]:
# extracting the best parameters
target_values = []
# for result in bopt_lgbm.res:
# target_values.append(result['target'])
# if result['target'] == max(target_values):
# best_params = result['params']

# print("Best Hyperparameters obtained are:\n")
# print(best_params)

<IPython.core.display.Javascript object>

**2-5-8** Training on optimized parameters.

In [31]:
params = {
    "learning_rate": 0.01,
    "n_estimators": 4750,
    "n_jobs": -1,
    "objective": "binary",
    "num_leaves": 39,
    "max_depth": 10,
    "boosting_type": "gbdt",
    "min_split_gain": 0.030820727751758883,
    "min_child_weight": 30.074868967458226,
    "min_child_samples": 31,
    "reg_alpha": 0.15663020002553255,
    "reg_lambda": 0.22503178038757748,
    "verbosity": -1,
    "metric": "auc",
    "seed": RANDOM_STATE,
    "feature_fraction": 0.1,
}

<IPython.core.display.Javascript object>

In [32]:
training_dataset = lgb.Dataset(X_train, label=y_train)
testing_dataset = lgb.Dataset(X_test, label=y_test)

<IPython.core.display.Javascript object>

In [33]:
clf = lgb.train(
    params,
    training_dataset,
    valid_sets=[training_dataset, testing_dataset],
)

[1]	training's auc: 0.71691	valid_1's auc: 0.710271
[2]	training's auc: 0.75442	valid_1's auc: 0.747887
[3]	training's auc: 0.750402	valid_1's auc: 0.742777
[4]	training's auc: 0.748001	valid_1's auc: 0.740192
[5]	training's auc: 0.745851	valid_1's auc: 0.738076
[6]	training's auc: 0.7445	valid_1's auc: 0.736491
[7]	training's auc: 0.742605	valid_1's auc: 0.734697
[8]	training's auc: 0.742075	valid_1's auc: 0.734509
[9]	training's auc: 0.742135	valid_1's auc: 0.734483
[10]	training's auc: 0.747942	valid_1's auc: 0.740503
[11]	training's auc: 0.746659	valid_1's auc: 0.739327
[12]	training's auc: 0.745487	valid_1's auc: 0.73825
[13]	training's auc: 0.74501	valid_1's auc: 0.737577
[14]	training's auc: 0.744366	valid_1's auc: 0.736924
[15]	training's auc: 0.744075	valid_1's auc: 0.736677
[16]	training's auc: 0.743421	valid_1's auc: 0.736059
[17]	training's auc: 0.742956	valid_1's auc: 0.735528
[18]	training's auc: 0.746151	valid_1's auc: 0.738805
[19]	training's auc: 0.746103	valid_1's auc

[152]	training's auc: 0.763973	valid_1's auc: 0.751868
[153]	training's auc: 0.764115	valid_1's auc: 0.751957
[154]	training's auc: 0.764188	valid_1's auc: 0.751988
[155]	training's auc: 0.764395	valid_1's auc: 0.752166
[156]	training's auc: 0.764468	valid_1's auc: 0.752212
[157]	training's auc: 0.764686	valid_1's auc: 0.75238
[158]	training's auc: 0.764782	valid_1's auc: 0.752429
[159]	training's auc: 0.764885	valid_1's auc: 0.75247
[160]	training's auc: 0.765054	valid_1's auc: 0.752604
[161]	training's auc: 0.765179	valid_1's auc: 0.752678
[162]	training's auc: 0.765268	valid_1's auc: 0.752711
[163]	training's auc: 0.765357	valid_1's auc: 0.752764
[164]	training's auc: 0.765397	valid_1's auc: 0.752777
[165]	training's auc: 0.765698	valid_1's auc: 0.753057
[166]	training's auc: 0.765998	valid_1's auc: 0.753335
[167]	training's auc: 0.766074	valid_1's auc: 0.753381
[168]	training's auc: 0.766171	valid_1's auc: 0.753461
[169]	training's auc: 0.766192	valid_1's auc: 0.753466
[170]	traini

[303]	training's auc: 0.780823	valid_1's auc: 0.763374
[304]	training's auc: 0.780881	valid_1's auc: 0.76338
[305]	training's auc: 0.780968	valid_1's auc: 0.763444
[306]	training's auc: 0.781145	valid_1's auc: 0.763598
[307]	training's auc: 0.781247	valid_1's auc: 0.763637
[308]	training's auc: 0.781327	valid_1's auc: 0.763686
[309]	training's auc: 0.781389	valid_1's auc: 0.763718
[310]	training's auc: 0.781475	valid_1's auc: 0.763771
[311]	training's auc: 0.781552	valid_1's auc: 0.763823
[312]	training's auc: 0.781717	valid_1's auc: 0.763958
[313]	training's auc: 0.781833	valid_1's auc: 0.764036
[314]	training's auc: 0.781983	valid_1's auc: 0.764166
[315]	training's auc: 0.782069	valid_1's auc: 0.764212
[316]	training's auc: 0.782225	valid_1's auc: 0.76435
[317]	training's auc: 0.782335	valid_1's auc: 0.764419
[318]	training's auc: 0.782432	valid_1's auc: 0.76449
[319]	training's auc: 0.782529	valid_1's auc: 0.764548
[320]	training's auc: 0.782605	valid_1's auc: 0.764595
[321]	trainin

[454]	training's auc: 0.793541	valid_1's auc: 0.771004
[455]	training's auc: 0.793596	valid_1's auc: 0.771027
[456]	training's auc: 0.793658	valid_1's auc: 0.771048
[457]	training's auc: 0.793715	valid_1's auc: 0.771065
[458]	training's auc: 0.793763	valid_1's auc: 0.771069
[459]	training's auc: 0.793815	valid_1's auc: 0.771087
[460]	training's auc: 0.793869	valid_1's auc: 0.771097
[461]	training's auc: 0.793954	valid_1's auc: 0.771156
[462]	training's auc: 0.794018	valid_1's auc: 0.771181
[463]	training's auc: 0.794092	valid_1's auc: 0.771219
[464]	training's auc: 0.794148	valid_1's auc: 0.771236
[465]	training's auc: 0.794239	valid_1's auc: 0.771301
[466]	training's auc: 0.79431	valid_1's auc: 0.771332
[467]	training's auc: 0.794361	valid_1's auc: 0.771358
[468]	training's auc: 0.794425	valid_1's auc: 0.771371
[469]	training's auc: 0.7945	valid_1's auc: 0.771405
[470]	training's auc: 0.794578	valid_1's auc: 0.771431
[471]	training's auc: 0.794654	valid_1's auc: 0.77147
[472]	training

[605]	training's auc: 0.802935	valid_1's auc: 0.775411
[606]	training's auc: 0.803002	valid_1's auc: 0.775432
[607]	training's auc: 0.803062	valid_1's auc: 0.775456
[608]	training's auc: 0.803112	valid_1's auc: 0.775465
[609]	training's auc: 0.803173	valid_1's auc: 0.775488
[610]	training's auc: 0.803239	valid_1's auc: 0.775514
[611]	training's auc: 0.803288	valid_1's auc: 0.77553
[612]	training's auc: 0.803334	valid_1's auc: 0.775548
[613]	training's auc: 0.803387	valid_1's auc: 0.775574
[614]	training's auc: 0.803441	valid_1's auc: 0.775595
[615]	training's auc: 0.803492	valid_1's auc: 0.775605
[616]	training's auc: 0.803531	valid_1's auc: 0.775604
[617]	training's auc: 0.803587	valid_1's auc: 0.775631
[618]	training's auc: 0.803653	valid_1's auc: 0.775687
[619]	training's auc: 0.803707	valid_1's auc: 0.775709
[620]	training's auc: 0.803764	valid_1's auc: 0.775748
[621]	training's auc: 0.803833	valid_1's auc: 0.775793
[622]	training's auc: 0.803886	valid_1's auc: 0.775811
[623]	train

...

[4183]	training's auc: 0.901591	valid_1's auc: 0.789435
[4184]	training's auc: 0.901618	valid_1's auc: 0.789432
[4185]	training's auc: 0.901642	valid_1's auc: 0.789435
[4186]	training's auc: 0.901657	valid_1's auc: 0.789435
[4187]	training's auc: 0.901679	valid_1's auc: 0.789437
[4188]	training's auc: 0.901682	valid_1's auc: 0.789437
[4189]	training's auc: 0.901696	valid_1's auc: 0.789441
[4190]	training's auc: 0.901711	valid_1's auc: 0.789443
[4191]	training's auc: 0.901718	valid_1's auc: 0.789446
[4192]	training's auc: 0.901738	valid_1's auc: 0.789435
[4193]	training's auc: 0.901761	valid_1's auc: 0.789429
[4194]	training's auc: 0.901782	valid_1's auc: 0.789427
[4195]	training's auc: 0.901801	valid_1's auc: 0.789421
[4196]	training's auc: 0.901826	valid_1's auc: 0.789418
[4197]	training's auc: 0.901848	valid_1's auc: 0.789414
[4198]	training's auc: 0.901867	valid_1's auc: 0.789415
[4199]	training's auc: 0.90189	valid_1's auc: 0.789416
[4200]	training's auc: 0.901911	valid_1's auc: 0.

[4331]	training's auc: 0.904105	valid_1's auc: 0.789299
[4332]	training's auc: 0.904118	valid_1's auc: 0.789297
[4333]	training's auc: 0.904133	valid_1's auc: 0.789298
[4334]	training's auc: 0.90414	valid_1's auc: 0.789297
[4335]	training's auc: 0.904156	valid_1's auc: 0.789299
[4336]	training's auc: 0.904173	valid_1's auc: 0.789296
[4337]	training's auc: 0.904178	valid_1's auc: 0.789299
[4338]	training's auc: 0.90419	valid_1's auc: 0.789303
[4339]	training's auc: 0.904208	valid_1's auc: 0.789305
[4340]	training's auc: 0.904225	valid_1's auc: 0.789309
[4341]	training's auc: 0.904246	valid_1's auc: 0.789314
[4342]	training's auc: 0.904265	valid_1's auc: 0.789314
[4343]	training's auc: 0.904283	valid_1's auc: 0.789312
[4344]	training's auc: 0.904306	valid_1's auc: 0.789321
[4345]	training's auc: 0.904318	valid_1's auc: 0.789322
[4346]	training's auc: 0.904335	valid_1's auc: 0.789322
[4347]	training's auc: 0.904356	valid_1's auc: 0.78933
[4348]	training's auc: 0.904378	valid_1's auc: 0.78

[4479]	training's auc: 0.906387	valid_1's auc: 0.789332
[4480]	training's auc: 0.906405	valid_1's auc: 0.789332
[4481]	training's auc: 0.906415	valid_1's auc: 0.789328
[4482]	training's auc: 0.906432	valid_1's auc: 0.789327
[4483]	training's auc: 0.906447	valid_1's auc: 0.789321
[4484]	training's auc: 0.906463	valid_1's auc: 0.789317
[4485]	training's auc: 0.90648	valid_1's auc: 0.789315
[4486]	training's auc: 0.9065	valid_1's auc: 0.789312
[4487]	training's auc: 0.906527	valid_1's auc: 0.789307
[4488]	training's auc: 0.906537	valid_1's auc: 0.78931
[4489]	training's auc: 0.906555	valid_1's auc: 0.789314
[4490]	training's auc: 0.906567	valid_1's auc: 0.789319
[4491]	training's auc: 0.906584	valid_1's auc: 0.789321
[4492]	training's auc: 0.906609	valid_1's auc: 0.789317
[4493]	training's auc: 0.906619	valid_1's auc: 0.789318
[4494]	training's auc: 0.906641	valid_1's auc: 0.789323
[4495]	training's auc: 0.906657	valid_1's auc: 0.789321
[4496]	training's auc: 0.90666	valid_1's auc: 0.7893

[4628]	training's auc: 0.908769	valid_1's auc: 0.789283
[4629]	training's auc: 0.908792	valid_1's auc: 0.789275
[4630]	training's auc: 0.908806	valid_1's auc: 0.789274
[4631]	training's auc: 0.908823	valid_1's auc: 0.789273
[4632]	training's auc: 0.908843	valid_1's auc: 0.789281
[4633]	training's auc: 0.908849	valid_1's auc: 0.789279
[4634]	training's auc: 0.908856	valid_1's auc: 0.789281
[4635]	training's auc: 0.908877	valid_1's auc: 0.789283
[4636]	training's auc: 0.908886	valid_1's auc: 0.789282
[4637]	training's auc: 0.908905	valid_1's auc: 0.789282
[4638]	training's auc: 0.908922	valid_1's auc: 0.789283
[4639]	training's auc: 0.908935	valid_1's auc: 0.789284
[4640]	training's auc: 0.908949	valid_1's auc: 0.789285
[4641]	training's auc: 0.908964	valid_1's auc: 0.789278
[4642]	training's auc: 0.90898	valid_1's auc: 0.789276
[4643]	training's auc: 0.908993	valid_1's auc: 0.789276
[4644]	training's auc: 0.909006	valid_1's auc: 0.789276
[4645]	training's auc: 0.90902	valid_1's auc: 0.7

<IPython.core.display.Javascript object>

In [34]:
y_proba = clf.predict(X_val)
to_csv(y_proba, "lgbm_WHO_WILL_WIN")

<IPython.core.display.Javascript object>

## 3. Conclusions

1. When dealing with many to one relationship inside csv files, it would be more efficient to extract not only AVG and COUNT but MIN, MAX as well.
2. Sometimes advanced imputing techniques like IterativeImputer, KNNImputer and MiceImputer could create bias inside of datasets and SimpleImputer with constant filling outperforms them.
3. Creating different features out of existing ones was the crucial part of the project.
4. Clustering as a feature had the highest correlation coeficient with the target.
5. When dealing with big datasets, it is important to choose fast models. LightGBM helped a lot.

![](proof.png)